# **Créditos por porcentaje de UMAS**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "tulancingo" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_tulancingo',
 'df_financiamientos_2020_tulancingo',
 'df_financiamientos_2021_tulancingo',
 'df_financiamientos_2022_tulancingo',
 'df_financiamientos_2023_tulancingo']

## 2022

In [2]:
finance = df_financiamientos_2022_tulancingo[['id','ingresos_rango']]
registros_0 = finance[finance['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

19
(316, 2)


In [4]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].astype(str)
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].replace({
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].replace({
    
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance

,id,ingresos_rango
0,1,2.61-4.00
1,2,0
2,3,≤ 2.60
3,4,9.01-12.00
4,5,2.61-4.00
...,...,...
311,312,> 12.00
312,313,6.01-9.00
313,314,> 12.00
314,315,> 12.00


In [5]:
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance)
conteo_ingresos['porcentajes'] = (conteo_ingresos['conteo'] / total_registros) * 100
# Eliminar la fila donde rango_ingresos es '0'
conteo_ingresos = conteo_ingresos[conteo_ingresos['rango_ingresos'] != '0']
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,2.61-4.00,70,22.151899
1,> 12.00,60,18.987342
2,6.01-9.00,60,18.987342
3,≤ 2.60,46,14.556962
4,4.01-6.00,38,12.025316
5,9.01-12.00,23,7.278481


In [6]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
3,≤ 2.60,46,14.556962
0,2.61-4.00,70,22.151899
4,4.01-6.00,38,12.025316
2,6.01-9.00,60,18.987342
5,9.01-12.00,23,7.278481
1,> 12.00,60,18.987342


In [7]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2022', carpeta='assets\graficas')
fig.show()

## 2023

In [8]:
finance = df_financiamientos_2023_tulancingo[['id','ingresos_rango']]
registros_0 = finance[finance['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

0
(45, 2)


In [9]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].astype(str)
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].replace({
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].replace({
    
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17096\2444930754.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['6' '6' '6' '3' '1' '4' '2' '2' '1' '4' '6' '3' '2' '5' '2' '6' '1' '3'
 '4' '6' '6' '4' '4' '3' '5' '1' '6' '6' '2' '4' '5' '6' '5' '5' '5' '3'
 '6' '5' '4' '2' '5' '6' '6' '2' '3']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



,id,ingresos_rango
0,1,> 12.00
1,2,> 12.00
2,3,> 12.00
3,4,4.01-6.00
4,5,≤ 2.60
5,6,6.01-9.00
6,7,2.61-4.00
7,8,2.61-4.00
8,9,≤ 2.60
9,10,6.01-9.00


In [10]:
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance)
conteo_ingresos['porcentajes'] = (conteo_ingresos['conteo'] / total_registros) * 100
# Eliminar la fila donde rango_ingresos es '0'
conteo_ingresos = conteo_ingresos[conteo_ingresos['rango_ingresos'] != '0']
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,13,28.888889
1,9.01-12.00,8,17.777778
2,2.61-4.00,7,15.555556
3,6.01-9.00,7,15.555556
4,4.01-6.00,6,13.333333
5,≤ 2.60,4,8.888889


In [11]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
5,≤ 2.60,4,8.888889
2,2.61-4.00,7,15.555556
4,4.01-6.00,6,13.333333
3,6.01-9.00,7,15.555556
1,9.01-12.00,8,17.777778
0,> 12.00,13,28.888889


In [12]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2023', carpeta='assets\graficas')
fig.show()